In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.3.0'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("musical_instruments").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://anaidu3.s3.amazonaws.com/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
toy_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep='\t', header=True, inferSchema=True)

In [ ]:
customer_count = toy_data_df.count()
customer_count

4864249

In [ ]:
review_id= toy_data_df.select(["review_id", "customer_id", "product_id", "product_parent","review_date"])
review_id.show()

In [ ]:
products_df= toy_data_df.select(["product_id", "product_title"])
products_df.show()

In [ ]:
from pyspark.sql.functions import row_number,lit
from pyspark.sql.window import Window


In [ ]:
w = Window().orderBy(lit('A'))
toy_data_df = toy_data_df.withColumn("row_num", row_number().over(w))
toy_data_df.show()

In [ ]:
toy_data_df = toy_data_df.withColumnRenamed("row_num","customer_count")
toy_data_df.show()

In [ ]:
customer_df= toy_data_df.select("customer_id", 'customer_count')
customer_df.show()

In [ ]:
vine= toy_data_df.select(["review_id", "star_rating", "helpful_votes", "total_votes","vine"])
vine.show()

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://endpoint:5432/toy_data"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write DataFrame to review_id table in RDS

review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to customers table in RDS

customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write dataframe to vine_table table in RDS

vine.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)